<a href="https://colab.research.google.com/github/VictorPelaez/genai_gazzete/blob/main/news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Example notebook

In [1]:
!pip install --upgrade newsapi-python transformers python-docx diffusers scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 69.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.3/36.3 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.0 MB/s eta 0:00:00
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184489 sha256=43acae1f2dc765454019a0570c6109ef03ba56ef0e4aab4d5715e03a2ce4fd9d
  Stored in directory: /root/.cache/pip/wheels/80/27/06/837436d4c3bd989b957a91679966f207bfd71d358d63a8194d
Successfully built python-docx
  Attempting uninstall: scipy
    Found existing installation: scipy 1.10.1
    Uninstalling scipy-1.1

In [2]:
!git clone https://github.com/VictorPelaez/genai_gazzete.git

Cloning into 'genai_gazzete'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 25 (delta 10), reused 15 (delta 4), pack-reused 0
Receiving objects: 100% (25/25), 8.34 KiB | 8.34 MiB/s, done.
Resolving deltas: 100% (10/10), done.


In [3]:
from genai_gazzete import utils
from newsapi import NewsApiClient

# -----------------------------------------------------
# Get news from fdate
# -----------------------------------------------------

fdate = '2023-08-04'

config = utils.readConfig()
d_config = dict(config.items('DEFAULT'))

newsapi = NewsApiClient(api_key=d_config['api_key'])
all_articles = newsapi.get_everything(q='generative ai llms',
                                      language='en',
                                      from_param=fdate,
                                      sort_by='relevancy')

print("#articles: ", len(all_articles["articles"]))

#articles:  25


## SUMMARIZATION MODEL

In [5]:
import pandas as pd
import time
from bs4 import BeautifulSoup
import requests
import re
from transformers import pipeline

import warnings
warnings.filterwarnings('ignore')

# -----------------------------------------------------
# Run Summarization model for all the articles
# -----------------------------------------------------

model_name = "sshleifer/distilbart-cnn-12-6" # other models: "sshleifer/bart-large-cnn" "sshleifer/distilbart-xsum-12-1" "google/flan-t5-base"
summarizer = pipeline('summarization', model="sshleifer/distilbart-cnn-12-6", device=0) # T4 GPU in Colab

dash_line = '-'.join('' for x in range(100))

## Data to feed
N = int(round(len(all_articles["articles"])*0.1, 0)) # Number of summaries to show, verbose
L = 3000 # Context size
df = pd.DataFrame(columns = ["source", "url", "title", "description", "len_text", "summary"])

for i, a in enumerate(all_articles["articles"]):

  start = time.time()
  print(dash_line)
  print('Example ', i + 1)
  print(dash_line)

  # read url HTLM web
  page = requests.get(a["url"])
  soup = BeautifulSoup(page.content, 'html.parser')
  result = soup.find_all(["p","i"]) # Slashdot is <i>
  ARTICLE = ""
  for part in result:
    if (part.get("class")==None) and len(ARTICLE)<L:
      ARTICLE = ARTICLE + " " + part.get_text()

  ARTICLE = utils.clear_article(ARTICLE)

  if len(ARTICLE)>0:
    summarized_article = summarizer(ARTICLE)[0]["summary_text"] # Inference summarization model
    summarized_article = re.sub(r'\s([?.!"](?:\s|$))', r'\1', summarized_article) # Remove whitespaces
  else:
    summarized_article = "empty"

  df = df.append({'source': a['source']['name'],
                  'url': a["url"],
                  'title': a["title"],
                  'description': a["description"],
                  'len_text': len(ARTICLE),
                  'summary': summarized_article}, ignore_index = True)

  # print samples
  if (i<N) and (len(ARTICLE)>0) :
    print(a['source']['name'], a["url"])
    print('Description:  ', a['description'])
    print('LLM Summary')
    print(df.iloc[i]["summary"])

  end = time.time()
  print(end - start)
  print(dash_line)
  print()

df = utils.remove_summaries(df)
print('Final number of articles: ' + str(len(df. index)))

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
Forbes https://www.forbes.com/sites/moorinsights/2023/08/04/aws-turbocharges-foundation-models-with-smart-ai-agents/
Description:   Vice President of AI & Quantum Computing, Paul Smith-Goodson discusses AWS's recently announced AI capability that adds key functionality to its foundation models.
LLM Summary
 aws recently announced an important ai capability called agents that adds key functionality to its foundation models before discussing agents in detail i will provide another overview of the amazon bedrock foundation models fms that will use these new features. The models are optimized and pretrained for conversations and content creation and because they run on an aws managed infrastructure the models can be scaled on services like ec2 and lambda.
1.757826566696167
---------

Your max_length is set to 142, but your input_length is only 4. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=2)


1.5273330211639404
---------------------------------------------------------------------------------------------------

---------------------------------------------------------------------------------------------------
Example  8
---------------------------------------------------------------------------------------------------
0.4091219902038574
---------------------------------------------------------------------------------------------------

---------------------------------------------------------------------------------------------------
Example  9
---------------------------------------------------------------------------------------------------
1.127666711807251
---------------------------------------------------------------------------------------------------

---------------------------------------------------------------------------------------------------
Example  10
---------------------------------------------------------------------------------------------------


Your max_length is set to 142, but your input_length is only 4. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=2)


1.5096440315246582
---------------------------------------------------------------------------------------------------

---------------------------------------------------------------------------------------------------
Example  11
---------------------------------------------------------------------------------------------------
1.6941845417022705
---------------------------------------------------------------------------------------------------

---------------------------------------------------------------------------------------------------
Example  12
---------------------------------------------------------------------------------------------------
3.315364122390747
---------------------------------------------------------------------------------------------------

---------------------------------------------------------------------------------------------------
Example  13
---------------------------------------------------------------------------------------------------
1.343

In [6]:
df.head(3)

,source,url,title,description,len_text,summary
0,Forbes,https://www.forbes.com/sites/moorinsights/2023...,AWS Turbocharges Foundation Models With Smart ...,"Vice President of AI & Quantum Computing, Paul...",2962,aws recently announced an important ai capabi...
1,Forbes,https://www.forbes.com/sites/forbestechcouncil...,Getting The Most Out Of AI: Businesses Need Sk...,The success of the prompt engineer will rely o...,3562,The last year has seen a meteoric rise in int...
2,Amazon.com,https://aws.amazon.com/about-aws/whats-new/202...,Announcing support for ml.p5 instances for Ama...,Amazon SageMaker training jobs now support ml....,1412,ml p5 instances powered by n Nvidia h100 chip...


# Image generation

In [7]:
import torch
from diffusers import StableDiffusionPipeline

model_id = "CompVis/stable-diffusion-v1-4"
device = "cuda"

pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to(device)

# prompt = "Google Assistant Is Getting a Major Upgrade, pastel colors, Victorian style"
# image = pipe(prompt).images[0]
# image.save("image.png")

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.


## Document Generation

In [8]:
import os
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from genai_gazzete import functions_doc
from docx import Document
from docx.shared import Inches
from docx.enum.text import WD_ALIGN_PARAGRAPH
from datetime import date

# -----------------------------------------------------
# document creation
# -----------------------------------------------------

today = date.today()
document = Document()
# document.add_heading("Check out the most popular articles about Generative AI and LLMs in a nutshell!", 0)
document.add_picture('/content/drive/MyDrive/LLMs/20230804_130845_0000.png', width=Inches(6), height=Inches(1.2))
last_paragraph = document.paragraphs[-1]
last_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
p = document.add_paragraph()
runner = p.add_run(fdate + " - " + str(today)).italic = True
last_paragraph = document.paragraphs[-1]
last_paragraph.alignment = WD_ALIGN_PARAGRAPH.RIGHT

for r in df.index:
  # 1. add title
  document.add_heading('[' + df["source"][r] + '] ' +df["title"][r], level=1)
  # 2. add link
  p = document.add_paragraph()
  functions_doc.add_hyperlink(p, 'Original article', df["url"][r])

  # 3. create image and add it
  prompt_style= " sci-fi style, pen and ink, pastel colors"
  image = pipe(df["title"][r] + prompt_style).images[0]
  image.save("image.png")
  document.add_picture('image.png', width=Inches(2), height=Inches(2))
  last_paragraph = document.paragraphs[-1]
  last_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

  # 4. add summary
  document.add_paragraph(df["summary"][r], style='Intense Quote')
  p = document.add_paragraph()
  p.paragraph_format.line_spacing = Inches(0.3)

file_name = "summarized_articles" + "_" + today.strftime('%m_%d_%Y') + ".docx"
document.save('/content/drive/MyDrive/LLMs/'+file_name)